In [1]:
import numpy as np
import pandas as pd

# **FCCN**

In [6]:
class FCNN:
    def __init__(self, layers_dim):
        self.W = [None]
        self.b = [None]
        self.layers_dim = layers_dim

        for i in range(1, len(layers_dim)):
            he = np.sqrt(2 / layers_dim[i-1])
            self.W.append(np.random.randn(layers_dim[i], layers_dim[i-1]) * he)
            self.b.append(np.zeros((layers_dim[i], 1)))

    def _relu(self, Z):
        return np.maximum(0, Z)

    def _sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))

    def forward(self, X):
        A = [X]
        Z = [None]

        for i in range(1, len(self.W)):
            Zi = np.dot(self.W[i], A[i-1]) + self.b[i]
            Z.append(Zi)

            if i == len(self.W) - 1:
                Ai = self._sigmoid(Zi)
            else:
                Ai = self._relu(Zi)

            A.append(Ai)
        return A, Z

    def backprop(self, Y, A, Z, lr):
        n = Y.shape[1]
        dZ = A[-1] - Y 

        for i in range(len(self.W) - 1, 0, -1):
            dW = np.dot(dZ, A[i-1].T) / n
            db = np.sum(dZ, axis=1, keepdims=True) / n

            if i > 1:
                dA_prev = np.dot(self.W[i].T, dZ)
                dZ = dA_prev * (Z[i-1] > 0)

            self.W[i] -= lr * dW
            self.b[i] -= lr * db

    def train(self, X_train, y_train, epochs=1000, lr=0.05):
        for i in range(epochs):
            A, Z = self.forward(X_train)
            self.backprop(y_train, A, Z, lr)
            
            if i % 100 == 0:
                c = 1e-15
                loss = -np.mean(y_train * np.log(A[-1] + c) + (1 - y_train) * np.log(1 - A[-1] + c))
                print(f"Iter {i}: Loss {loss:.4f}")

    def eval(self, X_test, y_test):
        A_test, _ = self.forward(X_test)
        predictions = (A_test[-1] > 0.5).astype(int)
        accuracy = np.mean(predictions == y_test) * 100
        print(f"Test Accuracy: {accuracy:.2f}%")

# **Implementation**

### **Loading**

In [3]:
columns = ['age', 'workclass', 'fnlwgt', 'education', 
           'education-num', 'marital-status', 'occupation', 'relationship', 
           'race', 'sex', 'capital-gain', 'capital-loss', 
           'hours-per-week', 'native-country', 'income']

train_df = pd.read_csv('adult.data', header=None, names=columns, na_values=' ?', skipinitialspace=True)
test_df = pd.read_csv('adult.test', header=None, names=columns, na_values=' ?', skipinitialspace=True, skiprows=1)

train_df['income'] = train_df['income'].replace({'>50K': 1, '<=50K': 0}).astype(int)

test_df['income'] = test_df['income'].str.replace('.', '', regex=False)
test_df['income'] = test_df['income'].replace({'>50K': 1, '<=50K': 0}).astype(int)

train_df = train_df.drop(['fnlwgt', 'education'], axis=1)
test_df = test_df.drop(['fnlwgt', 'education'], axis=1)

train_df = train_df.dropna()
test_df = test_df.dropna()

categ = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
train = pd.get_dummies(train_df, columns=categ, drop_first=True)
test = pd.get_dummies(test_df, columns=categ, drop_first=True)

test = test.reindex(columns=train.columns, fill_value=0)

feats = [col for col in train.columns if col != 'income']

C:\Users\sdhai\AppData\Local\Temp\ipykernel_1468\2873696779.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df['income'] = train_df['income'].replace({'>50K': 1, '<=50K': 0}).astype(int)
C:\Users\sdhai\AppData\Local\Temp\ipykernel_1468\2873696779.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df['income'] = test_df['income'].replace({'>50K': 1, '<=50K': 0}).astype(int)


### **Without Scaling**

In [4]:
print("--- Raw Data ---")

X_train_raw = train[feats].values.astype(float)
y_train_raw = train['income'].values.reshape(-1, 1)

X_test_raw = test[feats].values.astype(float)
y_test_raw = test['income'].values.reshape(-1, 1)

X_train_raw, y_train_raw = X_train_raw.T, y_train_raw.T
X_test_raw, y_test_raw = X_test_raw.T, y_test_raw.T

input_dim = X_train_raw.shape[0]
layer_dims = [input_dim, 64, 32, 1]

nn_raw = FCNN(layer_dims)
nn_raw.train(X_train_raw, y_train_raw, lr=0.01)

nn_raw.eval(X_test_raw, y_test_raw)

--- Raw Data ---


C:\Users\sdhai\AppData\Local\Temp\ipykernel_1468\1767189026.py:18: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-Z))


Iter 0: Loss 3.5207
Iter 100: Loss 0.5454
Iter 200: Loss 0.5413
Iter 300: Loss 0.5390
Iter 400: Loss 0.5373
Iter 500: Loss 0.5357
Iter 600: Loss 0.5342
Iter 700: Loss 0.5330
Iter 800: Loss 0.5319
Iter 900: Loss 0.5308
Test Accuracy: 76.46%


### **With Min-Max Scaling**

In [7]:
print("--- Min-Max Scaling ---")

X_train_mm = train[feats].values.astype(float)
y_train_mm = train['income'].values.reshape(-1, 1)

X_test_mm = test[feats].values.astype(float)
y_test_mm = test['income'].values.reshape(-1, 1)

mini = X_train_mm.min(axis=0)
maxi = X_train_mm.max(axis=0)

X_train_mm = (X_train_mm - mini) / (maxi - mini)
X_test_mm = (X_test_mm - mini) / (maxi - mini)

X_train_mm, y_train_mm = X_train_mm.T, y_train_mm.T
X_test_mm, y_test_mm = X_test_mm.T, y_test_mm.T

input_dim = X_train_mm.shape[0]
layer_dims = [input_dim, 64, 32, 1]

nn_mm = FCNN(layer_dims)
nn_mm.train(X_train_mm, y_train_mm, lr=0.1)

nn_mm.eval(X_test_mm, y_test_mm)

--- Min-Max Scaling ---
Iter 0: Loss 0.6252
Iter 100: Loss 0.3963
Iter 200: Loss 0.3732
Iter 300: Loss 0.3617
Iter 400: Loss 0.3541
Iter 500: Loss 0.3484
Iter 600: Loss 0.3440
Iter 700: Loss 0.3404
Iter 800: Loss 0.3376
Iter 900: Loss 0.3353
Test Accuracy: 84.42%
